In [1]:
import cvzone
from cvzone.HandTrackingModule import HandDetector
import cv2
import numpy as np
import pyautogui as control
import time
import mediapipe 
from cvzone.FPS import FPS
control.FAILSAFE =False

#define  detector
detector = HandDetector(staticMode=False,maxHands=2,modelComplexity=1,detectionCon=0.5, minTrackCon=0.5)                
##############variables#########
wCam, hCam = 640, 480
WScr ,HScr =control.size()
frameReduction = 100
plocationX,plocationY =0,0
clocationX,clocationY  =0 ,0 
smothingValue = 10
#######################
reader = FPS(avgCount=30)
cap = cv2.VideoCapture(0)
while True :
    _,frame = cap.read()
    #get all hands
    Hands ,frame= detector.findHands(frame,draw=False,flipType=True)
    if Hands :
        RightHand = Hands[0]
        RightHandLists=RightHand['lmList']
        #get needed tipes that we need for projects
        x1,y1 = RightHandLists[8][0:2]
        x2,y2 = RightHandLists[12][0:2]
        #get which finget is up
        cv2.rectangle(frame,(frameReduction,frameReduction),(wCam-frameReduction,hCam-frameReduction),(255,0,0),3)
        fingerOfRightHand = detector.fingersUp(RightHand)
        if fingerOfRightHand[1]==1 and fingerOfRightHand[2]==0:
            #start interpulate x,y from camer coridinates to full screen cordinates
            #x3 = np.interp(x1,(0,wCam),(0,WScr))
            #y3 = np.interp(y1,(0,hCam),(0,HScr))
            #strat change movment of pointer iside rctangle to waal screen
            x3 = np.interp(x1,(frameReduction,wCam-frameReduction),(0,WScr))
            y3 = np.interp(y1,(frameReduction,hCam-frameReduction),(0,HScr))
            #start smoothing
            clocationX =plocationX +(x3-plocationX)/smothingValue
            clocationY =plocationY +(y3-plocationY)/smothingValue
            cv2.circle(frame,(x1,y1),2,(255,0,0),cv2.FILLED)
            control.moveTo(WScr-clocationX,clocationY) 
            plocationX =clocationX
            clocationY = clocationY
           #click mouse
        if fingerOfRightHand[1]==1 and fingerOfRightHand[2]==1 :
             length, info, img = detector.findDistance(RightHandLists[8][0:2],RightHandLists[12][0:2] , frame, color=(255, 0, 255),scale=5) 
             if length<40 :
                 cv2.circle(frame,(info[4],info[5]),8,(0,255,255),cv2.FILLED)
                 control.click()
               #start right click  
        if fingerOfRightHand[0]==1 :
           control.rightClick()
           #start close camera 
        if fingerOfRightHand[0]==0 and fingerOfRightHand[1] ==0 and fingerOfRightHand[2]==0 and fingerOfRightHand[3] ==0 and fingerOfRightHand[4]==0 : 
            cap.release()
            break            
    imgPNG = cv2.imread("C:/Users/Geka/Desktop/update.png",cv2.IMREAD_UNCHANGED)
    imgOverlay = cvzone.overlayPNG(frame, imgPNG, pos=[3, 430])
    fps ,img = reader.update(frame,pos=(20,50),scale=1, thickness=2)
    cv2.imshow('ss',frame)
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 
